In [ ]:
#removing missing data
df.drop([1,2,3]) #drops rows by default
df.drop('A', axis=1)
df.dropna(subset=["B"])

In [ ]:
# Drop the Latitude and Longitude columns from volunteer
volunteer_cols = volunteer.drop(['Latitude','Longitude'], axis=1)

# Drop rows with missing category_desc values from volunteer_cols
volunteer_subset = volunteer_cols.dropna(subset=['category_desc'])

In [ ]:
#working with data types
# Convert the hits column to type int
volunteer["hits"] = volunteer["hits"].astype('int')

In [ ]:
# Split the dataset into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

knn = KNeighborsClassifier()

# Fit the knn model to the training data
knn.fit(X_train, y_train)

# Score the model on the test data
print(knn.score(X_test,y_test))

In [ ]:
#scaling
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
df_scaled = pd.DataFrame(scaler.fit_transform(df),
                        columns=df.columns)

In [ ]:
# Import StandardScaler
from sklearn.preprocessing import StandardScaler

# Create the scaler
scaler = StandardScaler()

# Subset the DataFrame you want to scale 
wine_subset = wine[['Ash', 'Alcalinity of ash', 'Magnesium']]

# Apply the scaler to wine_subset
wine_subset_scaled = scaler.fit_transform(wine_subset)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Instantiate a StandardScaler
scaler = StandardScaler()

# Scale the training and test features
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Fit the k-nearest neighbors model to the training data
knn.fit(X_train_scaled, y_train)

# Score the model on the test data
print(knn.score(X_test_scaled, y_test))

In [2]:
import pandas as pd
#encoding categorical variables
user=[1,2,3,4]
subscribed = ['y','n','n','y']
fav_color = ['blue','green','orange','green']
df = pd.DataFrame({'user': user,"subscribed":subscribed,'fav_color':fav_color})
df.head()

,user,subscribed,fav_color
0,1,y,blue
1,2,n,green
2,3,n,orange
3,4,y,green


In [5]:
#using pandas
df['sub_enc'] = df['subscribed'].apply(lambda val:1 if val=="y" else 0)

In [8]:
#encoding via sklearn
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
df['sub_enc_le'] = le.fit_transform(df['subscribed'])
df.head()

,user,subscribed,fav_color,sub_enc,sub_enc_le
0,1,y,blue,1,1
1,2,n,green,0,0
2,3,n,orange,0,0
3,4,y,green,1,1


In [9]:
#one hot encoding
pd.get_dummies(df['fav_color'])

,blue,green,orange
0,1,0,0
1,0,1,0
2,0,0,1
3,0,1,0


In [ ]:
# Set up the LabelEncoder object
enc = LabelEncoder()

# Apply the encoding to the "Accessible" column
hiking['Accessible_enc'] = enc.fit_transform(hiking['Accessible'])

# Compare the two columns
print(hiking[['Accessible', 'Accessible_enc']].head())

In [ ]:
#engineering numerical features
temps['mean'] = temps.loc[:,"day1":'day3'].mean(axis=1)
purchases['date_converted'] = pd.to_datetime(purchases['date'])
purchases['month'] = purchases.date_converted.dt.month


In [10]:
#engineering text features
import re
my_string = 'temperature:75.6 F'
temp = re.search('\d+\.\d+', my_string)
print(float(temp.group(0)))

75.6


In [ ]:
#vectorizing text
#TF/IDF:Vectorizes words based upon importance(Term Frequency, Inverse Document Frequency)
from sklearn.feature_extraction.text import TfidVectorizer
tfidf_vec = TfidVectorizer()
text_tfidf = tfidf_vec.fit_transform(documents)

In [ ]:
# Write a pattern to extract numbers and decimals
def return_mileage(length):
    
    # Search the text for matches
    mile = re.search('\d+\.\d+', length)
    
    # If a value is returned, use group(0) to return the found value
    if mile is not None:
        return float(mile.group(0))
        
# Apply the function to the Length column and take a look at both columns
hiking["Length_num"] = hiking['Length'].apply(return_mileage)
print(hiking[["Length", "Length_num"]].head())

In [ ]:
# Split the dataset according to the class distribution of category_desc
y = volunteer["category_desc"]
X_train, X_test, y_train, y_test = train_test_split(text_tfidf.toarray(),y , stratify=y, random_state=42)

# Fit the model to the training data
nb.fit(X_train, y_train)

# Print out the model's accuracy
print(nb.score(X_test, y_test))

In [ ]:
#removing redundant features
# Create a list of redundant column names to drop
to_drop = ["locality", "region", "category_desc", "vol_requests", "created_date"]

# Drop those columns from the dataset
volunteer_subset = volunteer.drop(to_drop, axis=1)

# Print out the head of volunteer_subset
print(volunteer_subset.head())

In [ ]:
#selecting features using text vectors
print(tfidf_vec.vocabulary_)

print(text_tfidf[3].data)
print(text_tfidf[3].indices)

#reverse data and indices
vocab = {v:k for k,v in tfidf_vec.vocabulary_.items()}

print(vocab)

In [ ]:
#zipping together
zipped_row = dict(zip(text_tfidf[3].indices,
                     text_tfidf[3].data))
print(zipped_row)

In [ ]:
#makking a function
def return_weights(vocab, vector, vector_index):
    
    zipped = dict(zip(vector[vector_index].indices,
                     vector[vector_index].data))
    
    return {vocab[i]:zipped[i] for i in vector[vector_index].indices}

print(return_weights(vocab,text_tfidf,3))

In [ ]:
# Add in the rest of the arguments
def return_weights(vocab, original_vocab, vector, vector_index, top_n):
    zipped = dict(zip(vector[vector_index].indices, vector[vector_index].data))
    
    # Transform that zipped dict into a series
    zipped_series = pd.Series({vocab[i]:zipped[i] for i in vector[vector_index].indices})
    
    # Sort the series to pull out the top n weighted words
    zipped_index = zipped_series.sort_values(ascending=False)[:top_n].index
    return [original_vocab[i] for i in zipped_index]

# Print out the weighted words
print(return_weights(vocab, tfidf_vec.vocabulary_, text_tfidf, 8,3))

In [ ]:
def words_to_filter(vocab, original_vocab, vector, top_n):
    filter_list = []
    for i in range(0, vector.shape[0]):
    
        # Call the return_weights function and extend filter_list
        filtered = return_weights(vocab, original_vocab, vector, i, top_n)
        filter_list.extend(filtered)
        
    # Return the list in a set, so we don't get duplicate word indices
    return set(filter_list)

# Call the function to get the list of word indices
filtered_words = words_to_filter(vocab, tfidf_vec.vocabulary_, text_tfidf, 3)

# Filter the columns in text_tfidf to only those in filtered_words
filtered_text = text_tfidf[:, list(filtered_words)]

In [ ]:
# Split the dataset according to the class distribution of category_desc
X_train, X_test, y_train, y_test = train_test_split(filtered_text.toarray(),y, stratify=y, random_state=42)

# Fit the model to the training data
nb.fit(X_train,y_train)

# Print out the model's accuracy
print(nb.score(X_test, y_test))

In [ ]:
#dimentionality reduction
from sklearn.decomposition import PCA
pca = PCA()
df_pca = pca.fit_transform(df)

print(df_pca)
print(pca.explained_variance_ratio_)

In [ ]:
# Instantiate a PCA object
pca = PCA()

# Define the features and labels from the wine dataset
X = wine.drop('Type', axis=1)
y = wine["Type"]

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Apply PCA to the wine dataset X vector
pca_X_train = pca.fit_transform(X_train)
pca_X_test = pca.transform(X_test)

# Look at the percentage of variance explained by the different components
print(pca.explained_variance_ratio_)

In [ ]:
# Fit knn to the training data
knn.fit(pca_X_train, y_train)

# Score knn on the test data and print it out
print(knn.score(pca_X_test,y_test))

In [ ]:
# Print the DataFrame info
print(ufo.info())

# Change the type of seconds to float
ufo["seconds"] = ufo["seconds"].astype('float')

# Change the date column to type datetime
ufo["date"] = pd.to_datetime(ufo["date"])

# Check the column types
print(ufo.info())

In [ ]:
# Count the missing values in the length_of_time, state, and type columns, in that order
print(ufo[['length_of_time', "state", "type"]].isna().sum())

# Drop rows where length_of_time, state, or type are missing
ufo_no_missing = ufo.dropna(subset=['length_of_time', "state", "type"], how='any',axis=0)

# Print out the shape of the new dataset
print(ufo_no_missing.shape)

In [ ]:
def return_minutes(time_string):

    # Search for numbers in time_string
    num = re.search("\d+",time_string )
    if num is not None:
        return int(num.group(0))
        
# Apply the extraction to the length_of_time column
ufo["minutes"] = ufo["length_of_time"].apply(return_minutes)

# Take a look at the head of both of the columns
print(ufo[['length_of_time','minutes']].head())

In [ ]:
# Check the variance of the seconds and minutes columns
print(ufo[['seconds','minutes']].var())

# Log normalize the seconds column
ufo["seconds_log"] = np.log(ufo['seconds'])

# Print out the variance of just the seconds_log column
print(ufo['seconds_log'].var())

In [ ]:
#FEATURE ENGINEERING
# Use pandas to encode us values as 1 and others as 0
ufo["country_enc"] = ufo["country"].apply(lambda x:1 if x=="us" else 0)

# Print the number of unique type values
print(len(ufo["type"].unique()))

# Create a one-hot encoded set of the type values
type_set = pd.get_dummies(ufo['type'])

# Concatenate this set back to the ufo DataFrame
ufo = pd.concat([ufo, type_set], axis=1)

In [ ]:
# Look at the first 5 rows of the date column
print(ufo['date'].head(5))

# Extract the month from the date column
ufo["month"] = ufo["date"].dt.month

# Extract the year from the date column
ufo["year"] = ufo["date"].dt.year

# Take a look at the head of all three columns
print(ufo[['date','month','year']])

In [ ]:
# Take a look at the head of the desc field
print(ufo['desc'].head())

# Instantiate the tfidf vectorizer object
vec = TfidfVectorizer()

# Fit and transform desc using vec
desc_tfidf = vec.fit_transform(ufo['desc'])

# Look at the number of columns and rows
print(desc_tfidf.shape)

In [ ]:
# Make a list of features to drop
to_drop = ['city', 'country', 'lat', 'long','state', 'date','recorded', 'seconds','minutes','desc','length_of_time']

# Drop those features
ufo_dropped = ufo.drop(to_drop, axis=1)

# Let's also filter some words out of the text vector we created
filtered_words = words_to_filter(vocab, vec.vocabulary_, desc_tfidf, 4)

In [ ]:
# Take a look at the features in the X set of data
print(X.columns)

# Split the X and y sets
X_train, X_test, y_train, y_test = train_test_split(X,y, stratify=y, random_state=42)

# Fit knn to the training sets
knn.fit(X_train,y_train)

# Print the score of knn on the test sets
print(knn.score(X_test,y_test))

In [ ]:
# Create `loan_data` subset: loan_data_subset
loan_data_subset = loan_data[['Credit Score','Annual Income','Loan Status']]

# Create train and test sets
trainingSet, testSet = train_test_split(loan_data_subset,test_size=0.2, random_state=123)

# Examine pairplots
plt.figure()
sns.pairplot(trainingSet, hue='Loan Status', palette='RdBu')
plt.show()

plt.figure()
sns.pairplot(testSet, hue='Loan Status', palette='RdBu')
plt.show()

In [ ]:
# Subset loan_data
cr_yrs = loan_data['Years of Credit History']

# Square root transform
cr_yrs_sqrt = boxcox(cr_yrs, lmbda=0.5)

# Histogram and kernel density estimate
plt.figure()
sns.distplot(cr_yrs_sqrt)
plt.show()